# !pip install tensorflow==1.15
!pip install stable-baselines[mpi]==2.10.2

In [3]:
import os
import math
import gym
import numpy as np
import pandas as pd
from gym import spaces
from stable_baselines import PPO1

/home/amirmahdi/anaconda3/envs/RL_Project2/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/home/amirmahdi/anaconda3/envs/RL_Project2/lib/python3.7/site-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


There are 6 possible log levels. Here it is a list of them starting from high verbosity to low:
* 1. Trace : detail info for tracing
* 2. Debug
* 3. Info : important but normal
* 4. Warn
* 5. Error
* 6. Fatal

Among all these 6 log levels, we have decided to eliminate **Error** and **Fatal** as these two are mostly related to runtime behavioir while the information that our model try to make its logging decisions only comes from static source code features. There fore it does not have enough knowledge to decide about logs with Error and Fatal level.


**Action space prespective:**


<img src= "images/action_space_implementations.png" alt="alt text"  />

For the first run of the implementation we will consider all of the for levels (**Combined**), and after this we implement the other way to compare them together.

For the first run implementatoin the **action space** takes **6** discrete values; 
* **"0"**: not_Log
* **"1"**: Trace
* **"2"**: Debug
* **"3"**: Info
* **"4"**: Warn
* **"5"**: IDK*

**Observatio space prespective:**

Observation space is going to be a vector (box) representing each function (rows of our dataset). It has shape of (1,14) it is a vector and we have 14 features to illusterate the fuction under study.

**Reward Function:**

<img src="images/reward_function.drawio.png" />

<img src="images/sigmoid1.png" 
 longdesc="https://www.wolframalpha.com/input?i=plot+%28%28sigmoid+function+0.3x+%29+minus+0.5+%29+multiply+by+2++from+-10+to+10" title="from WolframAlpha"/>

<img src="images/sigmoid2.png" />

The reward will be -4 if it was supposed to be logged but it was not, or vise versa. It will be 1 + sigmoid(features) if the chosen action by the model is the same as "_action_determiner"._ It wil be - abolute difference of determined log action and taken action. 

All these decisions might get changed or modified in testing and when we get to see the results.


**Log Level:**

<img src="images/which-log-level.png" title="from which log level paper"/>

<img src="images/which-log-level1.png" title="from which log level paper"/>

**Logic behind the _action_determiner:**

This function is developed based on one main deenition : " Most important performance bugs ( the ones that are more common to happend and more specific) will have **higher** **verbose**.

Based on this defenition the following performance bugs have been ordered based on their verbosity;( The higher position, the higher verbsity)

* 1. Synchronization
* 2. Race condition and dead Lock
* 3. Nested Loop level
* 4. API

We have came up we this defenition based on two main factors: 1. the importance of the performance bugs based on previous studies and our litreture review (imperical study) 2. Our features are more well defined to capture some of the performance bugs comparing to others, so that we have considered higher performance verbose for those performance bugs.

In [1]:
class source_code_env(gym.Env):
    metadata = {'render.modes': ['console']}
    
    number_of_actions = 6
    
    
    actions = {
        "NOT_LOG": 0,
        "TRACE": 1,
        "DEBUG": 2,
        "INFO": 3,
        "WARN": 4,
        "IDK": 5
    }
    
    features_for_reward_func = {
        "number_of_loops": 0,
        "nesting_level": 1,
        "input_dependant_level": 2,
        "number_of_defined_threads": 3,
        "number_of_started_threads": 4,
        "number_of_join_threads": 5,
        "number_of_defined_locks": 6,
        "number_of_acquired_locks_threads": 7,
        "number_of_released_locks": 8,
        "number_of_usage_of_extra": 9,
        "number_of_usage_of_values_list": 10,
        "number_of_usage_of_select_related": 11
    }
    
    
    def __init__(self, dataset):
        super(source_code_env, self).__init__()
        self.dataset = dataset.copy()
        self.action_space = spaces.Discrete(self.number_of_actions)
        self.observation_space = spaces.Box(low= dataset.min().min(), 
                                            high= dataset.max().max(), 
                                            shape=(len(dataset.columns),))
        self.t = 0
        self.t_limit = len(dataset)
        
        
    def _action_determiner(self, obs):
        """
        The implementation logic of assiging each performance bug to a certain log level
        is based on the knowledge from previous studies mentioned above, the how common and important they are and
        also our features capabalities.
        
        arguments:
        
        obs: is supposed to be a vector (a row of the pandas dataframe);
        
        """
        features_for_reward_func = self.features_for_reward_func
        actions = self.actions
        synchronization_features = obs[features_for_reward_func["number_of_defined_threads"]:
            features_for_reward_func["number_of_usage_of_extra"]]
        api_related_features = obs[features_for_reward_func["number_of_usage_of_extra"]:]
        determined_action = actions["IDK"] # Default action
        
        if (obs == 0).all(): # Checking if any of features has occured
            determined_action = actions["NOT_LOG"]
            return determined_action
        elif (synchronization_features > 0).any():
            determined_action = actions["TRACE"]
            
            obs_num_threads = obs[features_for_reward_func["number_of_defined_threads"]]
            obs_num_locks = obs[features_for_reward_func["number_of_defined_locks"]]
            obs_num_join = obs[features_for_reward_func["number_of_join_threads"]]
            race_condition_metric = obs_num_threads - obs_num_locks
            deadlock_metric = obs_num_threads - obs_num_join
            if race_condition_metric > 0 or deadlock_metric > 0: 
                determined_action = actions["DEBUG"]
            return determined_action
        else:
            nested_loop = obs[features_for_reward_func["nesting_level"]] > 0
            input_dependant = obs[features_for_reward_func["input_dependant_level"]] > 0
            if nested_loop or input_dependant:
                determined_action = actions["INFO"]
                return determined_action
            elif (api_related_features != 0).any():
                determined_action = actions["WARN"]
                return determined_action
        return determined_action
        
    
    def _sigmoid(self, x_coefficient, x):
        
        """
        arguments:
        
        x_coefficient: this is to control the slope of the Sigmoid function.
        x: input of the Sigmoid function.
        """
            
        x = x_coefficient * x
        return ((1/(1 + np.exp(-x))) - 0.5) * 2
    
    def _reward(self, obs, taken_action):
        
        """
        arguments:
        
        obs: is supposed to be a vector (a row of the pandas dataframe);
        taken action: is the action taken by model based on obs. Its value could be {0, 1 ... 4}.
        """
        """
        action_based_on_obs = None
        
        if(obs.sum() == 0):
            action_based_on_obs = actions("NOT_LOG")
        else:
            action_based_on_obs = actions("TRACE")"""
        
        determined_action = self._action_determiner(obs)
        
        if taken_action == determined_action: # if the answer is correct
            reward = 1 + self._sigmoid(0.3, np.sum(obs))
            return reward
        elif determined_action == 0 and taken_action != 0: # if it was not supposed to be logged but it logged
            reward = -4 #Worse scenario 0 - 4
            return reward
        elif determined_action != 0 and taken_action == 0: # if it was supposed to be logged but it did not log
            reward = -4 #Worse scenario 0 - 4
            return reward
        else: # wrong log level
            reward = -abs(taken_action - determined_action) # will be -3 at the worse
            return reward
        
    
    def reset(self):
        self.t = 0
        obs = self.dataset.iloc[self.t].astype(np.float32)
        return np.array(obs)
    
    def step(self, action):
        
        done = False
        
        obs = self.dataset.iloc[self.t].astype(np.float32) # getting current observation to compute reward
        reward = self._reward(obs, action)
        self.t += 1
        if self.t >= self.t_limit:
            done = True
            #obs = self.dataset.iloc[0].astype(np.float32) # getting updated observation after taken action
            obs = self.reset() # getting updated observation after taken action
        else:
            obs = self.dataset.iloc[self.t].astype(np.float32) # getting updated observation after taken action
        
        info={}
        
        return np.array(obs), reward, done, info

NameError: name 'gym' is not defined

In [ ]:
    def step(self, action):
        done = False
        obs = self.dataset.iloc[self.t].astype(np.float32)
        reward = self._reward(obs, action)
        self.t += 1
        if self.t >= self.t_limit:
            done = True
            obs = self.reset()
        else:
            obs = self.dataset.iloc[self.t].astype(np.float32)
        info={}
        return np.array(obs), reward, done, info

In [ ]:
def step(self, action):
        
        done = True
        self.t += 1
        
        if self.t < self.t_limit:
            done = False
        
            obs = self.dataset.iloc[self.t - 1].astype(np.float32) # getting current observation to compute reward
            reward = self._reward(obs, action)
            next_obs = self.dataset.iloc[self.t].astype(np.float32) # getting updated observation after taken action
        
            info={}
        
            return np.array(obs), reward, done, info

# Evaluation

### [1] Test

In [33]:
dataset = pd.read_csv("/home/amirmahdi/projects/Adaptive-Logging-system-git/code/data/dataframe_main.csv")
dataset = dataset.set_index("ID")
dataset = dataset.astype(int)

In [51]:
# Check the environment:
from stable_baselines.common.env_checker import check_env
env = source_code_env(dataset)
# If the environment don't follow the interface,        an error will be thrown
check_env(env, warn=True)

In [60]:
PPO1_model = PPO1('MlpPolicy', env, verbose=2)
PPO1_model.learn(total_timesteps= 36600)

********** Iteration 0 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
      0.00038 |      -0.01792 |    2088.82056 |      1.93e-05 |       1.79174
     -0.00257 |      -0.01792 |    2077.91992 |       0.00010 |       1.79166
     -0.00470 |      -0.01792 |    2066.78369 |       0.00026 |       1.79150
     -0.00719 |      -0.01791 |    2055.40576 |       0.00051 |       1.79126
Evaluating losses...
     -0.00857 |      -0.01791 |    2047.86987 |       0.00072 |       1.79105
----------------------------------
| EpThisIter      | 0            |
| EpisodesSoFar   | 0            |
| TimeElapsed     | 1.22         |
| TimestepsSoFar  | 256          |
| ev_tdlam_before | -0.0046      |
| loss_ent        | 1.7910523    |
| loss_kl         | 0.0007177076 |
| loss_pol_entpen | -0.017910521 |
| loss_pol_surr   | -0.008570649 |
| loss_vf_loss    | 2047.8699    |
----------------------------------
********** Iteration 1 ************
Optim

----------------------------------
| EpThisIter      | 0            |
| EpisodesSoFar   | 0            |
| TimeElapsed     | 6.17         |
| TimestepsSoFar  | 2304         |
| ev_tdlam_before | -0.00193     |
| loss_ent        | 1.4497163    |
| loss_kl         | 0.028811267  |
| loss_pol_entpen | -0.014497163 |
| loss_pol_surr   | -0.049058054 |
| loss_vf_loss    | 749.71533    |
----------------------------------
********** Iteration 9 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
      0.00048 |      -0.01449 |     534.74359 |      6.83e-05 |       1.44871
     -0.01240 |      -0.01413 |     521.06805 |       0.00163 |       1.41275
     -0.04082 |      -0.01330 |     508.42749 |       0.01131 |       1.32989
     -0.03944 |      -0.01257 |     496.98199 |       0.02634 |       1.25655
Evaluating losses...
     -0.04005 |      -0.01249 |     490.02679 |       0.02858 |       1.24900
----------------------------------
| EpTh

     -0.00132 |      -0.00165 |     273.01614 |      2.07e-05 |       0.16499
     -0.00658 |      -0.00155 |     266.23557 |       0.00026 |       0.15521
     -0.00674 |      -0.00148 |     259.23663 |       0.00068 |       0.14849
     -0.00690 |      -0.00144 |     252.14288 |       0.00105 |       0.14442
Evaluating losses...
     -0.00699 |      -0.00143 |     247.74203 |       0.00125 |       0.14275
-----------------------------------
| EpThisIter      | 0             |
| EpisodesSoFar   | 0             |
| TimeElapsed     | 11.9          |
| TimestepsSoFar  | 4608          |
| ev_tdlam_before | -0.000146     |
| loss_ent        | 0.14274754    |
| loss_kl         | 0.0012458339  |
| loss_pol_entpen | -0.0014274754 |
| loss_pol_surr   | -0.006986685  |
| loss_vf_loss    | 247.74203     |
-----------------------------------
********** Iteration 18 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     -0.00055 |      -0.0010

********** Iteration 26 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     3.05e-05 |      -0.00056 |     279.57004 |      4.08e-06 |       0.05551
     -0.00177 |      -0.00054 |     269.02130 |      2.44e-05 |       0.05415
     -0.00197 |      -0.00052 |     258.63321 |      7.05e-05 |       0.05239
     -0.00206 |      -0.00051 |     248.59172 |       0.00013 |       0.05065
Evaluating losses...
     -0.00220 |      -0.00050 |     242.34239 |       0.00017 |       0.04980
-----------------------------------
| EpThisIter      | 0             |
| EpisodesSoFar   | 0             |
| TimeElapsed     | 17.3          |
| TimestepsSoFar  | 6912          |
| ev_tdlam_before | -0.0481       |
| loss_ent        | 0.04980295    |
| loss_kl         | 0.00016595183 |
| loss_pol_entpen | -0.0004980295 |
| loss_pol_surr   | -0.0022043511 |
| loss_vf_loss    | 242.34239     |
-----------------------------------
********** Iteration 27 ****

     -0.00060 |      -0.00030 |     203.83542 |      2.31e-05 |       0.03024
Evaluating losses...
     -0.00072 |      -0.00030 |     201.30542 |      3.75e-05 |       0.02950
------------------------------------
| EpThisIter      | 0              |
| EpisodesSoFar   | 0              |
| TimeElapsed     | 22.3           |
| TimestepsSoFar  | 8960           |
| ev_tdlam_before | 0.00708        |
| loss_ent        | 0.029501768    |
| loss_kl         | 3.752351e-05   |
| loss_pol_entpen | -0.00029501767 |
| loss_pol_surr   | -0.0007247403  |
| loss_vf_loss    | 201.30542      |
------------------------------------
********** Iteration 35 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     7.26e-08 |      -0.00044 |     211.34218 |      3.90e-07 |       0.04372
     -0.00019 |      -0.00044 |     207.05203 |      4.03e-06 |       0.04392
     -0.00082 |      -0.00046 |     202.70869 |      4.67e-05 |       0.04566
     -0.00175 | 

...

     -0.00048 |      -0.00023 |      81.84380 |      4.59e-06 |       0.02288
Evaluating losses...
     -0.00050 |      -0.00023 |      81.12625 |      6.15e-06 |       0.02281
------------------------------------
| EpThisIter      | 0              |
| EpisodesSoFar   | 0              |
| TimeElapsed     | 64.1           |
| TimestepsSoFar  | 26368          |
| ev_tdlam_before | 0.00427        |
| loss_ent        | 0.02281329     |
| loss_kl         | 6.145052e-06   |
| loss_pol_entpen | -0.00022813288 |
| loss_pol_surr   | -0.0004971577  |
| loss_vf_loss    | 81.12625       |
------------------------------------
********** Iteration 103 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
    -2.26e-06 |      -0.00025 |     104.36066 |      1.90e-07 |       0.02468
     -0.00016 |      -0.00025 |     103.05017 |      7.21e-07 |       0.02456
     -0.00021 |      -0.00024 |     101.72766 |      1.81e-06 |       0.02443
     -0.00023 |

********** Iteration 111 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     3.37e-07 |     -7.99e-05 |      96.61245 |      3.34e-08 |       0.00799
     1.84e-07 |     -7.97e-05 |      95.60997 |      3.61e-08 |       0.00797
     6.61e-08 |     -7.97e-05 |      94.59792 |      1.86e-08 |       0.00797
    -1.12e-08 |     -7.96e-05 |      93.57634 |      2.11e-08 |       0.00796
Evaluating losses...
    -6.89e-08 |     -7.96e-05 |      92.95287 |      8.58e-08 |       0.00796
-----------------------------------
| EpThisIter      | 0             |
| EpisodesSoFar   | 0             |
| TimeElapsed     | 69.8          |
| TimestepsSoFar  | 28672         |
| ev_tdlam_before | 0.00312       |
| loss_ent        | 0.007962128   |
| loss_kl         | 8.5811436e-08 |
| loss_pol_entpen | -7.962128e-05 |
| loss_pol_surr   | -6.891787e-08 |
| loss_vf_loss    | 92.952866     |
-----------------------------------
********** Iteration 112 **

     -0.00017 |      -0.00012 |      64.35500 |      7.74e-07 |       0.01186
Evaluating losses...
     -0.00017 |      -0.00012 |      64.03848 |      8.88e-07 |       0.01186
------------------------------------
| EpThisIter      | 0              |
| EpisodesSoFar   | 0              |
| TimeElapsed     | 74.7           |
| TimestepsSoFar  | 30720          |
| ev_tdlam_before | -0.00069       |
| loss_ent        | 0.011861717    |
| loss_kl         | 8.876214e-07   |
| loss_pol_entpen | -0.00011861718 |
| loss_pol_surr   | -0.00016809441 |
| loss_vf_loss    | 64.038475      |
------------------------------------
********** Iteration 120 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
    -9.08e-05 |      -0.00045 |      67.39813 |      1.34e-07 |       0.04525
     -0.00057 |      -0.00045 |      66.87146 |      3.05e-06 |       0.04509
     -0.00098 |      -0.00045 |      66.34431 |      3.47e-05 |       0.04463
     -0.00103 |

********** Iteration 128 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
    -8.38e-09 |      -0.00020 |      76.39561 |     -9.35e-08 |       0.02027
    -2.54e-05 |      -0.00020 |      75.97720 |     -5.70e-08 |       0.02026
    -7.61e-05 |      -0.00020 |      75.56776 |      7.49e-07 |       0.02023
     -0.00011 |      -0.00020 |      75.16478 |      2.11e-06 |       0.02021
Evaluating losses...
     -0.00012 |      -0.00020 |      74.92058 |      2.95e-06 |       0.02019
------------------------------------
| EpThisIter      | 0              |
| EpisodesSoFar   | 0              |
| TimeElapsed     | 80.1           |
| TimestepsSoFar  | 33024          |
| ev_tdlam_before | 0.0128         |
| loss_ent        | 0.020190991    |
| loss_kl         | 2.9508315e-06  |
| loss_pol_entpen | -0.00020190992 |
| loss_pol_surr   | -0.00012001023 |
| loss_vf_loss    | 74.92058       |
------------------------------------
********** Iter

    -5.86e-05 |      -0.00018 |      66.90054 |      5.75e-07 |       0.01778
Evaluating losses...
    -5.87e-05 |      -0.00018 |      66.79913 |      8.36e-07 |       0.01779
------------------------------------
| EpThisIter      | 0              |
| EpisodesSoFar   | 0              |
| TimeElapsed     | 85.2           |
| TimestepsSoFar  | 35072          |
| ev_tdlam_before | -0.000482      |
| loss_ent        | 0.017789261    |
| loss_kl         | 8.361047e-07   |
| loss_pol_entpen | -0.00017789262 |
| loss_pol_surr   | -5.8675185e-05 |
| loss_vf_loss    | 66.79913       |
------------------------------------
********** Iteration 137 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
    -5.26e-06 |      -0.00025 |      90.02191 |     -4.58e-08 |       0.02467
    -2.87e-05 |      -0.00025 |      89.84583 |      6.94e-08 |       0.02471
    -3.92e-05 |      -0.00025 |      89.66078 |      4.14e-07 |       0.02474
    -3.95e-05 |

### [2] Main

In [28]:
def RLM_training(model, dataset, sample_freq):
    main_env = model.env
    iteration= math.floor(len(dataset) / sample_freq)
    
    accumulative_rewards = []
    for _ in range(10):
        start = 0
        end = sample_freq
        for i in range(iteration):
            print(f"************** ITERATION {i} ***************")
            train_env = source_code_env(dataset.iloc[start:end])
            start += sample_freq
            end += sample_freq
            model.env = train_env
            model.learn(total_timesteps= sample_freq)

            # Making predictions on test set 
            obs = main_env.reset()
            sum_rewards = 0
            dones=0
            while dones==0:
                action, _states = model.predict(obs)
                obs, rewards, dones, info = main_env.step(action)
                sum_rewards = sum_rewards + rewards

            print(sum_rewards)
            accumulative_rewards.append(sum_rewards)
            #model.setup_model()
    model.set_env(main_env)
    model.setup_model()
    return accumulative_rewards

In [7]:
dataset = pd.read_csv("/home/amirmahdi/projects/Adaptive-Logging-system-git/code/data/dataframe_main.csv")
dataset = dataset.set_index("ID")
dataset = dataset.astype(int)

In [29]:
env = source_code_env(dataset)
PPO1_model = PPO1('MlpPolicy', env, verbose=0)

2023-08-22 14:36:28.270218: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-22 14:36:28.270870: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce GTX 850M major: 5 minor: 0 memoryClockRate(GHz): 0.9015
pciBusID: 0000:0a:00.0
2023-08-22 14:36:28.271356: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.0'; dlerror: libcudart.so.10.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/amirmahdi/anaconda3/envs/RL_Project2/lib/python3.7/site-packages/cv2/../../lib64:
2023-08-22 14:36:28.271744: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcublas.so.10.0'; dlerror: libcublas.so.10.0: cannot open shared object file: No such file or direct

In [30]:
RLM_training(PPO1_model, dataset, 1000)

************** ITERATION 0 ***************
-105892.43333233181
************** ITERATION 1 ***************
-93366.86445800232
************** ITERATION 2 ***************
-52859.55859076892
************** ITERATION 3 ***************
-241.21294408099027
************** ITERATION 4 ***************
25041.244123070937
************** ITERATION 5 ***************
29334.184808858437
************** ITERATION 6 ***************
31816.80949102468
************** ITERATION 7 ***************
32761.255658405396
************** ITERATION 8 ***************
33243.951508952254
************** ITERATION 9 ***************
33352.36642821269
************** ITERATION 10 ***************
33456.03556007492
************** ITERATION 11 ***************
33533.11899358783
************** ITERATION 12 ***************
33548.38190711744
************** ITERATION 13 ***************
32820.339882977496
************** ITERATION 14 ***************
33368.39426957003
************** ITERATION 15 ***************
33745.44470004284
*******

KeyboardInterrupt: 

In [ ]:
# Lists to store rewards for learning curve
episode_rewards = []

# Train the agent and collect rewards
total_timesteps = 1000
for _ in range(total_timesteps):
    model.learn(total_timesteps=env.spec.max_episode_steps)
    mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=1)
    episode_rewards.append(mean_reward)

# Future Work:

* Ading Function names
* Adding file names 
* TF-IDF
* Read/Write
* API MIsuses scenarios
* GP
* Trying different implementaton of reward function
    * Adding "NO_ACTION" as one of the actions
    * Mathematical implementation
    * Negative Reward
    * Normalization of the data
* Adding more source code
* Which Log level paper:

<img src="images/which-log-level-f.png" />

<img src="images/which-log-level-f1.png" />

* IDK 

# Test

In [ ]:
class LogEnvironment(gym.Env):
    metadata = {'render.modes': ['console']}
    
    
    def __init__(self,dataset, high = None, low = None):
        super(LogEnvironment, self).__init__()
        self.rewards = []
        self.dataset = dataset
        self.states = np.array(dataset.iloc[:,:-1])
        self.state = 0
        self.predict_classes = []
        
        n_actions = 4
        self.action_space = spaces.Discrete(n_actions)
        if high is None:
            low = np.array(dataset.iloc[:,:-1].min()).astype(np.float32)
            high = np.array(dataset.iloc[:,:-1].max()).astype(np.float32)
            self.observation_space = spaces.Box(low=low, high=high, dtype=np.float32)
        else:
            self.observation_space = spaces.Box(low=low, high=high, dtype=np.float32)
        
    
    def reset(self):
        self.state = 0
        LogEnvironment.predicts.append(self.predict_classes)
        self.predict_classes = []
        return self.states[self.state].astype(np.float32)
    
    def step(self, action):
        if action == self.NoLog:
            self.predict_classes.append(0)
        elif action == self.Log:
            self.predict_classes.append(1)
        else:
            raise ValueError("Received invalid action={} which is not part of the action space".format(action))
        
        done = bool(self.state >= len(self.classes)-1)
        
        
        
        reward = 1 if self.predict_classes[self.state] == self.classes[self.state] else 0
        self.rewards.append(reward)
        
        self.state += 0 if done else 1
        
        info = {}
        
        return self.states[self.state].astype(np.float32), reward, done, info
        

In [37]:
class test:
    d = []
    p = 1
    def __init__(self, dataset):
        self.dataset = dataset.copy()
        test.d.append(p)
        
    def fun(self):
        self.dataset.append("a")

In [38]:
lst = ["Amir"]
cls = test(lst)

NameError: name 'p' is not defined

In [36]:
lst

['Amir']

In [35]:
cls.dataset

['Amir', 'a']

In [33]:
cls.d

['b']

In [6]:
features_for_reward_func = {
        "number_of_loops": 0,
        "nesting_level": 1,
        "input_dependant_level": 2,
        "number_of_defined_threads": 3,
        "number_of_started_threads": 4,
        "number_of_join_threads": 5,
        "number_of_defined_locks": 6,
        "number_of_acquired_locks_threads": 7,
        "number_of_released_locks": 8,
        "number_of_usage_of_extra": 9,
        "number_of_usage_of_values_list": 10,
        "number_of_usage_of_select_related": 11
    }
features_for_reward_func["number_of_loops"]

0

In [3]:
a = 1 > 0
print(a)

True


In [7]:
def _sigmoid(x_coefficient, x):
        
        """
        arguments:
        
        x_coefficient: this is to control the slope of the Sigmoid function.
        x: input of the Sigmoid function.
        """
            
        x = x_coefficient * x
        return ((1/(1 + np.exp(-x))) - 0.5) * 2
    
_sigmoid(0.35, 6)

0.7818063576087741

In [24]:
def _action_determiner(obs):
        """
        The implementation logic of assiging each performance bug to a certain log level
        is based on the knowledge from previous studies mentioned above, the how common and important they are and
        also our features capabalities.
        
        arguments:
        
        obs: is supposed to be a vector (a row of the pandas dataframe);
        
        """
        synchronization_features = obs[features_for_reward_func["number_of_defined_threads"]:
            features_for_reward_func["number_of_usage_of_extra"]]
        api_related_features = obs[features_for_reward_func["number_of_usage_of_extra"]:]
        determined_action = actions["IDK"] # Default action
        
        if (obs == 0).all(): # Checking if any of features has occured
            determined_action = actions["NOT_LOG"]
            return determined_action
        elif (synchronization_features > 0).any():
            determined_action = actions["TRACE"]
            
            obs_num_threads = obs[features_for_reward_func["number_of_defined_threads"]]
            obs_num_locks = obs[features_for_reward_func["number_of_defined_locks"]]
            obs_num_join = obs[features_for_reward_func["number_of_join_threads"]]
            race_condition_metric = obs_num_threads - obs_num_locks
            deadlock_metric = obs_num_threads - obs_num_join
            if race_condition_metric > 0 or deadlock_metric > 0: 
                determined_action = actions["DEBUG"]
            return determined_action
        else:
            nested_loop = obs[features_for_reward_func["nesting_level"]] > 0
            input_dependant = obs[features_for_reward_func["input_dependant_level"]] > 0
            if nested_loop or input_dependant:
                determined_action = actions["INFO"]
                return determined_action
            elif (api_related_features != 0).any():
                determined_action = actions["WARN"]
                return determined_action
            else:
                determined_action = 5
        return determined_action

In [14]:
actions = {
    "NOT_LOG": 0,
    "TRACE": 1,
    "DEBUG": 2,
    "INFO": 3,
    "WARN": 4,
    "IDK": 5
}

features_for_reward_func = {
    "number_of_loops": 0,
    "nesting_level": 1,
    "input_dependant_level": 2,
    "number_of_defined_threads": 3,
    "number_of_started_threads": 4,
    "number_of_join_threads": 5,
    "number_of_defined_locks": 6,
    "number_of_acquired_locks_threads": 7,
    "number_of_released_locks": 8,
    "number_of_usage_of_extra": 9,
    "number_of_usage_of_values_list": 10,
    "number_of_usage_of_select_related": 11
}
df = pd.read_csv("data/dataframe.csv").set_index("ID")


number_of_loops                      0.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of_released_locks             0.0
number_of_usage_of_extra             0.0
number_of_usage_of_values_list       0.0
number_of_usage_of_select_related    0.0
Name: ee2c971f20132061d7e749267b166730ff734a7c, dtype: float32


In [25]:
for i in range(len(df)):
    obs = df.iloc[i].astype(np.float32)
    
    if _action_determiner(obs) != 0:
        print(obs)
        print("ACTION: ", _action_determiner(obs))
        print("------------------------------------------")

number_of_loops                      1.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of_released_locks             0.0
number_of_usage_of_extra             0.0
number_of_usage_of_values_list       0.0
number_of_usage_of_select_related    0.0
Name: dd15cd5b22997fef1b6efa5ffa71d6ce84de30c5, dtype: float32
ACTION:  5
------------------------------------------
number_of_loops                      1.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of_released_locks             0.0
number_of_usage_of_ext

ACTION:  5
------------------------------------------
number_of_loops                      1.0
nested_loop_level                    2.0
loop_input_dependent_level           2.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of_released_locks             0.0
number_of_usage_of_extra             0.0
number_of_usage_of_values_list       0.0
number_of_usage_of_select_related    0.0
Name: 0a25ba5991316bdda4a9b3abcee2106016df28a0, dtype: float32
ACTION:  3
------------------------------------------
number_of_loops                      1.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of

ACTION:  3
------------------------------------------
number_of_loops                      1.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of_released_locks             0.0
number_of_usage_of_extra             0.0
number_of_usage_of_values_list       0.0
number_of_usage_of_select_related    0.0
Name: 97418507bc2fc7d55435bc3ca888b6b73c00df6e, dtype: float32
ACTION:  5
------------------------------------------
number_of_loops                      2.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of

ACTION:  3
------------------------------------------
number_of_loops                      2.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of_released_locks             0.0
number_of_usage_of_extra             0.0
number_of_usage_of_values_list       0.0
number_of_usage_of_select_related    0.0
Name: 1925f793093581c47a5c6b2e414506b1cbae280d, dtype: float32
ACTION:  5
------------------------------------------
number_of_loops                      1.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of

In [26]:
for i in range(len(df)):
    print(df.iloc[i] if (df.iloc[i] != 0).any() else "-------------------")

-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
number_of_loops                      1.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of_released_locks             0.0
number_of_usage_of_extra             0.0
number_of_usage_of_values_list       0.0
number_of_usage_of_select_related    0.0
Name: dd15cd5b22997fef1b6efa5ffa71d6ce84de30c5, dtype: float64
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
number_of_loops                      1.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
nu

-------------------
-------------------
number_of_loops                      2.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of_released_locks             0.0
number_of_usage_of_extra             0.0
number_of_usage_of_values_list       0.0
number_of_usage_of_select_related    0.0
Name: e540cdd1328b2b21e29a95405c301b9313b7c346, dtype: float64
number_of_loops                      1.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of_released_locks             0.0
number_of_usage_of_extra            

number_of_loops                      1.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of_released_locks             0.0
number_of_usage_of_extra             0.0
number_of_usage_of_values_list       0.0
number_of_usage_of_select_related    0.0
Name: 1925f793093581c47a5c6b2e414506b1cbae280d, dtype: float64
-------------------
number_of_loops                      1.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of_released_locks             0.0
number_of_usage_of_extra             0.0
number_of_usage